In [1]:
import os
os.chdir('/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/graph_var')

from graph_var.graph_var.utils import load_graph_from_pkl, read_gfa_line_by_line, node_complement
from graph_var.graph_var.graph import PangenomeGraph
from graph_var.graph_var.evaluating_functions import *
import pandas as pd

version = 'v1'

graph_obj_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Graph_objs_{version}"
raw_vcf_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/VCFs_chr"
graph_vcf_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/VCFs_{version}"

ref_tree_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/reference_tree_gfa_{version}"
gfa_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/chromosome_gfa_{version}"
snarl_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/chr_snarls_{version}"
bubble_summary_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Bubble_summary_{version}"

var_summary_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Stats_chr_{version}"
data_vis_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data_visualization_{version}"

region_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Region_files"

mode = 'AT'
wavevcf = True
exclude_terminus = True

In [2]:
chr_list = list(range(1,23))
# chr_list = [22]

SNP count in HLA-A large insertion

In [7]:
hla_insertion = ("61637118", "61637368")

chr6_AT = pd.read_csv(f"{bubble_summary_dir}/bubble_variant_counts_chr6_AT.tsv", sep='\t')
hla_insertion_var = eval(chr6_AT[chr6_AT['Bubble'] == str(hla_insertion)]['Within'].iloc[0])

chr6_var_dict = {}

for row in tqdm(read_vcf_line_by_line(f"{graph_vcf_dir}/graph_chr6_no_terminus.vcf")):
    ID = extract_bubble_ids(row['ID'], symbol=True)
    if ID not in hla_insertion_var:
        continue
    chr6_var_dict[ID] = row

1740551it [00:31, 55135.61it/s]


In [10]:
snp = 0
offlinear_snp = 0

for id, data in chr6_var_dict.items():
    info =  get_info_dict(data['INFO'])
    if info['VT'] == 'SNP':
        snp += 1
        if int(info['DR'].split(',')[1]) != 0:
            offlinear_snp += 1

In [11]:
snp, offlinear_snp

(62, 62)

Check the max length of node in chr22

In [2]:
chr22_graph = load_graph_from_pkl(f"{graph_obj_dir}/chr22.pkl.gz", compressed=True)

In [5]:
max_len = 0
count = 0
for node, data in chr22_graph.nodes(data=True):
    length =  len(data['sequence'])
    if length > max_len:
        max_len = length
    if length > 1024:
        count += 1
print(max_len, count)

10500493 1484


Check the reason of two rawvcf only SNPs

In [3]:
def get_walks(gfa_path):
    walk_list = []
    for parts in read_gfa_line_by_line(gfa_path):
        if parts[0] != 'W':
            continue
        sample_name = parts[2]
        walk = parts[3]
        walk_list.append((sample_name, walk))
    return walk_list

In [4]:
chr6_graph = load_graph_from_pkl(f"{graph_obj_dir}/chr6.pkl.gz", compressed=True)
chr7_graph = load_graph_from_pkl(f"{graph_obj_dir}/chr7.pkl.gz", compressed=True)

In [5]:
example1_nodes = ['61846503_+', '61846504_+', '61846505_-', '61846506_-', '61846507_+']
example2_nodes = ['67791427_+', '67791428_+', '67791429_-', '67791430_-', '67791431_+']

In [6]:
chr6_walks = get_walks(f"{gfa_dir}/chr6.gfa")
chr7_walks = get_walks(f"{gfa_dir}/chr7.gfa")

In [11]:
print("Positive subgraph:")
for node in example1_nodes:
    print(node, list(chr6_graph.neighbors(node)), list(chr6_graph.predecessors(node)))
print("Negative subgraph:")
for node in example1_nodes[::-1]:
    node = node_complement(node)
    print(node, list(chr6_graph.neighbors(node)), list(chr6_graph.predecessors(node)))

Positive subgraph:
61846503_+ ['61846506_-', '61846504_+'] ['61846501_+', '61846502_-']
61846504_+ ['61846507_+', '61846505_-'] ['61846503_+']
61846505_- ['61846506_-'] ['61846504_+']
61846506_- ['61846507_+', '-_terminus_+'] ['61846503_+', '61846505_-']
61846507_+ ['61846508_+', '61846509_-'] ['61846504_+', '61846506_-']
Negative subgraph:
61846507_- ['61846504_-', '61846506_+'] ['61846508_-', '61846509_+']
61846506_+ ['61846503_-', '61846505_+'] ['61846507_-', '-_terminus_-']
61846505_+ ['61846504_-'] ['61846506_+']
61846504_- ['61846503_-'] ['61846507_-', '61846505_+']
61846503_- ['61846501_-', '61846502_+'] ['61846506_+', '61846504_-']


In [41]:
unique_positive_paths = set()
unique_negative_paths = set()
for sample, walk in chr6_walks:
    pos_start = example1_nodes[0]
    neg_start = node_complement(example1_nodes[-1])
    if pos_start in set(walk):
        start_idx_pos = walk.index(pos_start)
        unique_positive_paths.add('|'.join(walk[start_idx_pos:start_idx_pos+5]))
    if neg_start in set(walk):
        start_idx_neg = walk.index(neg_start)
        unique_negative_paths.add('|'.join(walk[start_idx_neg:start_idx_neg+5]))
    if walk[0] == "61846506_+":
        unique_negative_paths.add('|'.join(walk[:5]))
    if walk[-1] == "61846506_-":
        unique_positive_paths.add('|'.join(walk[-5:]))

In [42]:
unique_positive_paths, unique_negative_paths

({'61846503_+|61846504_+|61846507_+|61846508_+|61846510_+',
  '61846503_+|61846506_-|61846507_+|61846508_+|61846510_+'},
 {'61846506_+|61846505_+|61846504_-|61846503_-|61846501_-',
  '61846507_-|61846504_-|61846503_-|61846501_-|61846500_-',
  '61846507_-|61846506_+|61846503_-|61846502_+|61846500_-'})

In [19]:
print("Positive subgraph:")
for node in example2_nodes:
    print(node, list(chr7_graph.neighbors(node)), list(chr7_graph.predecessors(node)))
print("Negative subgraph:")
for node in example2_nodes[::-1]:
    node = node_complement(node)
    print(node, list(chr7_graph.neighbors(node)), list(chr7_graph.predecessors(node)))

Positive subgraph:
67791427_+ ['67791428_+', '67791430_-'] ['67791425_+', '67791426_-']
67791428_+ ['67791431_+', '67791429_-'] ['67791427_+', '+_terminus_+']
67791429_- ['67791430_-'] ['67791428_+']
67791430_- ['67791431_+'] ['67791427_+', '67791429_-']
67791431_+ ['67791432_+', '67791433_-'] ['67791428_+', '67791430_-']
Negative subgraph:
67791431_- ['67791428_-', '67791430_+'] ['67791432_-', '67791433_+']
67791430_+ ['67791427_-', '67791429_+'] ['67791431_-']
67791429_+ ['67791428_-'] ['67791430_+']
67791428_- ['67791427_-', '+_terminus_-'] ['67791431_-', '67791429_+']
67791427_- ['67791425_-', '67791426_+'] ['67791428_-', '67791430_+']


In [46]:
unique_positive_paths = set()
unique_negative_paths = set()
for sample, walk in chr7_walks:
    pos_start = example2_nodes[0]
    neg_start = node_complement(example2_nodes[-1])
    if pos_start in set(walk):
        start_idx_pos = walk.index(pos_start)
        unique_positive_paths.add('|'.join(walk[start_idx_pos:start_idx_pos+5]))
    if neg_start in set(walk):
        start_idx_neg = walk.index(neg_start)
        unique_negative_paths.add('|'.join(walk[start_idx_neg:start_idx_neg+5]))
    if walk[0] == "67791428_+":
        unique_positive_paths.add('|'.join(walk[:5]))
    if walk[-1] == "67791428_-":
        unique_negative_paths.add('|'.join(walk[-5:]))

In [47]:
unique_positive_paths, unique_negative_paths

({'67791427_+|67791428_+|67791431_+|67791432_+|67791434_+',
  '67791427_+|67791428_+|67791431_+|67791433_-|67791434_+',
  '67791427_+|67791430_-|67791431_+|67791432_+|67791434_+'},
 {'67791431_-|67791428_-|67791427_-|67791425_-|67791424_-',
  '67791431_-|67791430_+|67791427_-|67791425_-|67791424_-',
  '67791431_-|67791430_+|67791427_-|67791426_+|67791424_-',
  '67791431_-|67791430_+|67791429_+|67791428_-',
  '67791432_-|67791431_-|67791430_+|67791429_+|67791428_-'})

In [45]:
for sample, walk in chr7_walks:
    if len(walk) < 10:
        print(walk)

Length of non-GRCh38 sequence

In [3]:
total_length = 0

total_length_linear = 0
total_length_offlinear = 0

for i in tqdm(chr_list):
    G = load_graph_from_pkl(f"{graph_obj_dir}/chr{i}.pkl.gz", compressed=True)
    for node, data in G.nodes(data=True):
        if data['direction'] == 1 and data['on_reference_path'] == 1:
            total_length_linear += len(data['sequence'])
        if data['direction'] == 1 and data['on_reference_path'] == 0:
            total_length_offlinear += len(data['sequence'])
        total_length += len(data['sequence'])

  0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 22/22 [51:03<00:00, 139.23s/it] 


In [4]:
total_length, total_length_linear, total_length_offlinear

(6008516378, 2875001522, 129256667)

In [5]:
(total_length_linear + total_length_offlinear) * 2

6008516378

Number of nodes, edges, walks for the whole gfa graph

In [7]:
num_nodes = 0
num_edges = 0
hap_walk_dicts = dict()

In [8]:
for parts in read_gfa_line_by_line("/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/entire_gfa/hprc-v1.1-mc-grch38.gfa.gz", compressed=True):
    if parts[0] == 'S':
        binode, sequence = parts[1], parts[2]
        num_nodes += 1
    elif parts[0] == 'L':
        biedge = parts[1]
        node1 = biedge[0] + '_' + biedge[2]
        node2 = biedge[1] + '_' + biedge[3]
        num_edges += 1
    elif parts[0] == 'W':
        hit_reference = parts[1]
        sample_name = parts[2]
        walk = parts[3]

        hap_walk_dicts[sample_name] = hap_walk_dicts.get(sample_name, 0) + 1        

In [9]:
print(num_nodes, num_edges, 
      len(hap_walk_dicts), 
      min(hap_walk_dicts.values()), 
      max(hap_walk_dicts.values()),
      sum(hap_walk_dicts.values())/len(hap_walk_dicts))

80069733 110938345 90 113 566 286.3333333333333


INDEL count comparison (Pantree VCF, Raw VCF, Wave VCF)

In [2]:
def recursion_dict():
    return defaultdict(recursion_dict)

In [6]:
indel_count_per_sample = recursion_dict()
sample_list = None
entry_counts = [0,0,0]

for chr in tqdm(chr_list):
    graph_vcf_path = f"{graph_vcf_dir}/graph_chr{chr}{'_no_terminus' if exclude_terminus else ''}.vcf"
    rawvcf_path = f"{raw_vcf_dir}/hprc-v1.1-mc-grch38.raw_chr{chr}.vcf"
    vcfwave_path = f"{raw_vcf_dir}/hprc-v1.1-mc-grch38.vcfbub.a100k.wave_chr{chr}.vcf"

    for row in read_vcf_line_by_line(graph_vcf_path):
        samples = list(row.keys())[9:]

        info_dict = get_info_dict(row['INFO'])
        VT = info_dict['VT']
        ref = row['REF'] if row['REF'] != '.' else info_dict['NR']
        alt = row['ALT']
        length = (len(ref) if ref != '.' else 0) + (len(alt) if alt != '.' else 0)

        linear = int(info_dict['DR'].split(',')[1]) == 0

        if VT not in {'INS', 'DEL'} or length < 50 or not linear:
            continue
        entry_counts[0] += 1
        for sample in samples:
            entry = row[sample]
            gt = entry.split(':')[0]
            count = len({idx for idx in gt.split('|') if idx not in {'.', '0'}})
            assert count <= 2
            if 'pantree' in indel_count_per_sample and sample in indel_count_per_sample['pantree']:
                indel_count_per_sample['pantree'][sample] += count
            else:
                indel_count_per_sample['pantree'][sample] = count
    
    for row in read_vcf_line_by_line(rawvcf_path):
        samples = list(row.keys())[9:]

        if sample_list is None:
            sample_list = samples

        ref = row['REF']
        alts = row['ALT'].split(',')

        for idx, alt in enumerate(alts):
            length = len(ref) + len(alt)

            IND = False
            if (len(ref) <= 1 and len(alt) > 1) or (len(alt) <= 1 and len(ref) > 1):
                IND = True

            if not IND or length < 50:
                continue
            entry_counts[1] += 1
            for sample in samples:
                entry = row[sample]
                gt = entry.split(':')[0]
                count = len({idx for idx in gt.split('|') if idx not in {'.', '0'}})
                assert count <= 2
                if 'rawvcf' in indel_count_per_sample and sample in indel_count_per_sample['rawvcf']:
                    indel_count_per_sample['rawvcf'][sample] += count
                else:
                    indel_count_per_sample['rawvcf'][sample] = count

    for row in read_vcf_line_by_line(vcfwave_path):
        samples = list(row.keys())[9:]

        ref = row['REF']
        alts = row['ALT'].split(',')

        for idx, alt in enumerate(alts):
            length = len(ref) + len(alt)

            IND = False
            if (len(ref) <= 1 and len(alt) > 1) or (len(alt) <= 1 and len(ref) > 1):
                IND = True

            if not IND or length < 50:
                continue
            entry_counts[2] += 1
            for sample in samples:
                entry = row[sample]
                gt = entry.split(':')[0]
                count = len({idx for idx in gt.split('|') if idx not in {'.', '0'}})
                assert count <= 2
                if 'vcfwave' in indel_count_per_sample and sample in indel_count_per_sample['vcfwave']:
                    indel_count_per_sample['vcfwave'][sample] += count
                else:
                    indel_count_per_sample['vcfwave'][sample] = count

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:50<00:00, 50.38s/it]


In [7]:
entry_counts

[2926, 4348, 10913]

In [8]:
df = pd.DataFrame(columns=['Sample', 
                           'Pantree > 50bp INDEL alleles',
                           'Raw VCF > 50bp INDEL alleles',
                           'Vcfwave > 50bp INDEL alleles'])

for sample in sample_list:
    df.loc[len(df)] = [sample, 
     indel_count_per_sample['pantree'][sample], 
     indel_count_per_sample['rawvcf'][sample], 
     indel_count_per_sample['vcfwave'][sample]]

In [9]:
df

,Sample,Pantree > 50bp INDEL alleles,Raw VCF > 50bp INDEL alleles,Vcfwave > 50bp INDEL alleles
0,CHM13,471,2159,423
1,HG00438,681,3447,711
2,HG00621,618,3521,663
3,HG00673,625,3452,660
4,HG00733,687,3746,736
5,HG00735,630,3566,685
6,HG00741,715,3953,771
7,HG01071,663,3567,697
8,HG01106,671,3566,694
9,HG01109,785,4123,836


In [59]:
df.to_csv("/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data_visualization_v1/distinct_indel_gt_50_count_per_sample.csv")

Maximum Level in Snarl

In [6]:
max_level = 0

for i in tqdm(chr_list):
    ac_df = pd.read_csv(f"{bubble_summary_dir}/bubble_allele_summary_chr{i}.tsv", sep='\t')
    for row in ac_df.itertuples(index=False):
        level = int(row.Level)
        if level > max_level:
            max_level = level

print(max_level)

100%|██████████| 22/22 [01:30<00:00,  4.10s/it]

28


Count of biallelic (1 variant) and multi-allelic superbubbles (>= 2 variants)

In [17]:
biallelic_count = 0
multiallelic_count = 0

for chr in tqdm(chr_list):
    summary_path = f"{bubble_summary_dir}/bubble_variant_counts_chr{chr}_AT.tsv"
    AT_csv = pd.read_csv(summary_path, sep='\t')

    biallelic_count += AT_csv['Within_count'].apply(lambda x: x == 1).sum()
    multiallelic_count += AT_csv['Within_count'].apply(lambda x: x > 1).sum()

print(biallelic_count, multiallelic_count)

100%|██████████| 22/22 [01:39<00:00,  4.50s/it]

22700968 1923161


Statistics for triallelic-bubble categories

In [4]:
bubble_triallelic_path = f"{bubble_summary_dir}/bubble_triallelic_all_chr.tsv"
bubble_triallelic_df = pd.read_csv(bubble_triallelic_path, sep='\t')

In [6]:
bubble_triallelic_df['Bubble_type'].value_counts()

Bubble_type
interlocking           463169
overlapping            397291
properly_triallelic    133104
nested                  43808
Name: count, dtype: int64

Analysis 1: median max_allele_length of high allelic superbubbles

In [95]:
max_allele_list = []

for i in tqdm(range(1,23)):
    bubble_allele_summary_path = f"{bubble_summary_dir}/bubble_allele_summary_chr{i}.tsv"
    bubble_type_path = f"{bubble_summary_dir}/superbubble_type_chr{i}.tsv"

    bubble_ac_df = pd.read_csv(bubble_allele_summary_path, sep='\t')
    bubble_type_df = pd.read_csv(bubble_type_path, sep='\t')

    filter_bool = bubble_ac_df['Level'].apply(lambda x: int(x) == 0) & bubble_ac_df['num_variants'].apply(lambda x: int(x) >= 10)

    bubble_10_more_vars = set(bubble_ac_df['Bubble_id'][filter_bool])
    bubble_10_more_vars = set(map(lambda x: tuple(sorted(eval(x), key=lambda y: (len(y), y))), bubble_10_more_vars))
    max_allele_list.extend(bubble_type_df['Max_allele_length'][bubble_type_df['Bubble'].apply(lambda x: tuple(sorted(eval(x), key=lambda y: (len(y), y)))).isin(bubble_10_more_vars)].to_list())

  0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 22/22 [10:21<00:00, 28.23s/it]


In [96]:
len(max_allele_list)

104160

In [97]:
import numpy as np

np.median(max_allele_list)

22.0

Analysis 2: Details within the high allelic superbubbles

In [29]:
high_allelic_superbubble_dict = {}

for i in tqdm(range(1,23)):
    bubble_ac_path = f"{bubble_summary_dir}/bubble_allele_summary_chr{i}.tsv"
    bubble_AT_path = f"{bubble_summary_dir}/bubble_variant_counts_chr{i}_AT.tsv"
    vcf_path = f"{graph_vcf_dir}/graph_chr{i}_no_terminus.vcf"

    bubble_ac_df = pd.read_csv(bubble_ac_path, sep='\t')
    bubble_ac_dict = {}

    for row in bubble_ac_df.itertuples(index=False):
        if int(row.Level) != 0: 
            continue
        bubble = tuple(sorted(eval(row.Bubble)))

        ac = int(row.raw_vcf_allele_count) if row.raw_vcf_allele_count != '.' else 0

        if ac >= 10:
            bubble_ac_dict[bubble] = ac

    bubble_AT_df = pd.read_csv(bubble_AT_path, sep='\t')

    var_set = set()
    bubble_within = {}
    for row in bubble_AT_df.itertuples(index=False):
        bubble = tuple(sorted(eval(row.Bubble)))
        if bubble not in bubble_ac_dict:
            continue
        var_within = eval(row.Within)
        var_set = var_set.union(var_within)
        bubble_within[bubble] = var_within

    var_dict = {}
    for row in read_vcf_line_by_line(vcf_path):
        var_id = extract_bubble_ids(row['ID'], symbol=True)
        if not var_id in var_set:
            continue
        info_dict = get_info_dict(row['INFO'])
        POS = int(row['POS'])
        VT = info_dict['VT']
        ref = row['REF'] if row['REF'] != '.' else info_dict['NR']
        alt = row['ALT']
        allele_length = len(ref) + len(alt)

        info_dict.pop('NR')
        var_dict[var_id] = (POS, allele_length, info_dict)

    for k, v in bubble_within.items():
        high_allelic_superbubble_dict[k] = list(map(lambda x: (x, var_dict[x]), v))


100%|██████████| 22/22 [24:20<00:00, 66.41s/it]


In [54]:
len(high_allelic_superbubble_dict)

95632

In [ ]:

for i in tqdm(range(1,23)):
    bubble_AT_path = f"{bubble_summary_dir}/bubble_variant_counts_chr{i}_AT.tsv"

In [53]:
total = 0
for k, v in high_allelic_superbubble_dict.items():
    total += len(v)
print(total)

4504608


In [39]:
insertion_1kbp = 0
deletion_50bp = 0
both = 0

reference_var = 0
reference_var_insertion_1kbp = 0

non_reference_var = 0
non_reference_var_insertion_1kbp = 0

In [40]:
bubble_with_large_insertion = set()
bubble_with_large_deletion = set()

for k, v in high_allelic_superbubble_dict.items():
    large_var_set = set()
    for var_id, var_data in v:
        allele_length = var_data[1]
        VT = var_data[2]['VT']
        DR_v = int(var_data[2]['DR'].split(',')[1])
        if DR_v != 0:
            non_reference_var += 1
        else:
            reference_var += 1
        if VT == 'INS' and allele_length >= 1000:
            large_var_set.add('INS')
        if VT == 'DEL' and allele_length >= 50:
            large_var_set.add('DEL')
    if 'INS' in large_var_set and 'DEL' in large_var_set:
        insertion_1kbp += 1
        deletion_50bp += 1
        both += 1
        bubble_with_large_insertion.add(k)
        bubble_with_large_deletion.add(k)
    elif 'INS' in large_var_set:
        insertion_1kbp += 1
        bubble_with_large_insertion.add(k)
    elif 'DEL' in large_var_set:
        deletion_50bp += 1
        bubble_with_large_deletion.add(k)

for k, v in high_allelic_superbubble_dict.items():
    if not k in bubble_with_large_insertion:
        continue
    for var_id, var_data in v:
        allele_length = var_data[1]
        VT = var_data[2]['VT']
        DR_v = int(var_data[2]['DR'].split(',')[1])
        if DR_v != 0:
            non_reference_var_insertion_1kbp += 1
        else:
            reference_var_insertion_1kbp += 1

In [41]:
print("Number of superbubbles with >= 1kbp insertion:", insertion_1kbp)
print("Number of superbubbles with >= 50bp deletion:", deletion_50bp)
print("Number of superbubbles with both:", both)

print("Number of reference variants:", reference_var)
print("Number of reference variants in superbubbles with >= 1kbp insertion:", reference_var_insertion_1kbp)

print("Number of non-reference variants:", non_reference_var)
print("Number of non-reference variants in superbubbles with >= 1kbp insertion:", non_reference_var_insertion_1kbp)

Number of superbubbles with >= 1kbp insertion: 2022
Number of superbubbles with >= 50bp deletion: 16242
Number of superbubbles with both: 1410
Number of reference variants: 2216930
Number of reference variants in superbubbles with >= 1kbp insertion: 718875
Number of non-reference variants: 2287678
Number of non-reference variants in superbubbles with >= 1kbp insertion: 1306030


In [45]:
insertion_1kbp/len(high_allelic_superbubble_dict), non_reference_var_insertion_1kbp/3472412

(0.021143550276058222, 0.3761160829993676)

In [46]:
off_linear_var_counts = []

for bubble in bubble_with_large_insertion:
    off_linear_var_count = 0
    vars = high_allelic_superbubble_dict[bubble]
    for var in vars:
        linear = int(var[1][2]['DR'].split(',')[1]) == 0
        if not linear:
            off_linear_var_count += 1
    off_linear_var_counts.append(off_linear_var_count)

In [49]:
sum(off_linear_var_counts)/len(off_linear_var_counts)

645.9099901088032

Non-superbubble SNPs

In [11]:
chr_list = list(range(1,23))

In [12]:
non_superbubble_snp_count = [0, 0, 0]

for i in tqdm(chr_list):
    graph_vcf_path = f"{graph_vcf_dir}/graph_chr{i}{'_no_terminus' if exclude_terminus else ''}.vcf"
    bubble_AT_path = f"{bubble_summary_dir}/bubble_variant_counts_chr{i}_AT.tsv"

    var_dict = {}

    for row in read_vcf_line_by_line(graph_vcf_path):
        edge = extract_bubble_ids(row['ID'], symbol=True)
        info_dict = get_info_dict(row['INFO'])
        info_dict['POS'] = int(row['POS'])

        var_dict[edge] = info_dict
    
    bubble_AT_df = pd.read_csv(bubble_AT_path, sep='\t')
    bubble_within_dict = {}

    for row in bubble_AT_df.itertuples(index=False):
        bubble = tuple(sorted(eval(row.Bubble)))

        if int(row.Level) == 0:
            continue

        within = eval(row.Within)

        for var in list(within):
            if var_dict[var]['VT'] == 'SNP':
                non_superbubble_snp_count[0] += 1
            
            linear = int(var_dict[var]['DR'].split(',')[1]) == 0
            if var_dict[var]['VT'] == 'SNP' and linear:
                non_superbubble_snp_count[1] += 1
            
            if var_dict[var]['VT'] == 'SNP' and not linear:
                non_superbubble_snp_count[2] += 1

  0%|          | 0/22 [00:27<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
non_superbubble_snp_count, non_superbubble_snp_count[2]/non_superbubble_snp_count[0]

([117683, 55228, 62455], 0.5307053695096148)